## Notebook to test the vae model on smaller dataset

### load and samle the data

```python

In [ ]:
import sys

sys.path.append("../../")

import utils.dataloader as dataloader

set = dataloader.load_pandas()
set = dataloader.get_correct_data(set)
set = dataloader.sample_data(set, 100)

train, test, val = dataloader.split_data(set)
len(train)

/home/bendm/machine_learning/project/majority-vote-outlier-detection/models/VAE/../../utils/dataloader.py:19: DtypeWarning: Columns (91,94,209,213) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(data_path, delimiter=";").iloc[:, -3:]


60

In [4]:
from Variational_AutoEncoder import VariationalAutoEncoder as VAE
from datetime import datetime

import torch
from torch.utils.tensorboard import SummaryWriter

In [ ]:
batch_size = 4
epochs = 10
input_dim = 10
hidden_dim = 5
latent_dim = 2

test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=True)
train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VAE(input_dim=784, hidden_dim=hidden_dim, latent_dim=latent_dim).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
writer = SummaryWriter(f'runs/mnist/vae_{datetime.now().strftime("%Y%m%d-%H%M%S")}')